# 畳み込みニューラルネットワークを構築しよう
---

深層学習ライブラリ Keras を使って手書き数字のデータセット MNIST を分類するニューラルネットワークを構築しましょう。

**はじめに**
- for文やwhile文の利用は明示的な利用指示がない場所での利用は避けてください。

**本演習の目的**
- CNN（畳み込みニューラルネットワーク）を構築する

## ライブラリのインストール

まずはじめに、本演習で利用するライブラリのインポートを行います。

- [numpy](http://www.numpy.org) 数値計算を行うための基本パッケージの公式ドキュメント

In [1]:
import tensorflow as tf
import keras

import numpy as np
import matplotlib.pyplot as plt

# 各レイヤーで使用するものをインポートします。
from keras.layers import Dense, Flatten, Activation
from keras.models import Sequential
from keras.layers import Conv2D

/home/zero2one/miniconda3/envs/zcar01/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zero2one/miniconda3/envs/zcar01/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zero2one/miniconda3/envs/zcar01/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zero2one/miniconda3/envs/zcar01

In [2]:
print(tf.__version__)
print(keras.__version__)

1.1.0
1.2.0


## データの読み込み

In [3]:
# mnistデータセットを準備します。
from keras.datasets import mnist

img_rows, img_cols = 28, 28

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train, X_test = X_train / 255.0, X_test / 255.0

from keras.utils.np_utils import to_categorical
y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

ここで4つの変数に訓練データとテストデータを格納しました。

* x_train: 訓練データ
* y_train: 訓練データ用の正解ラベル
* x_test: テストデータ
* y_test: テストデータ用の正解ラベル

それぞれの配列を確認しましょう。

In [4]:
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

X_train: (60000, 28, 28, 1)
y_train: (60000, 10)
X_test: (10000, 28, 28, 1)
y_test: (10000, 10)


ここで、X_trainのshapeの並びに注目してみると`(60000, 28, 28, 1)`となっています。左から（バッチサイズ、行、列、チャネル）となっている点に気をつけてください。このようなフォーマットをチャンネルラストと言います。逆に（バッチサイズ、チャネル、行、列）をチャンネルファーストと言います。この配列は畳み込み層に使う配列に合わせて変更をしています。

## 畳み込みニューラルネットワークのモデルを構築

Sequential モデルを構築します。

<img src="./img/cnn.png">

2次元の畳み込みレイヤー`Conv2D`を第1層に使うときは`input_shape`を指定します。今回のデータセットはチャンネルラストで用意したので、`input_shape`は、`input_shape = (rows, cols, channels)`となります。

**[Conv2Dのドキュメント](https://keras.io/ja/layers/convolutional/)**

```py
keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
```


では、実際にモデルを構築してみましょう。

In [5]:
# Sequential モデルを構築します。
model = Sequential()
model.add(Conv2D(32, 3, 3, activation='relu', input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

## モデルのコンパイル

In [6]:
# 評価指標（metrics）として accuracy を追加しています。
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## モデルの確認

In [7]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 26, 26, 32)    320         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 24, 24, 64)    18496       convolution2d_1[0][0]            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 36864)         0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 128)           4718720     flatten_1[0][0]                  
___________________________________________________________________________________________

## モデルを学習させてみよう

In [ ]:
# エポック=10で学習します。
epochs=10
fit = model.fit(X_train, y_train, batch_size=128, nb_epoch=epochs, verbose=1, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 404s - loss: 0.1514 - acc: 0.9538 - val_loss: 0.0435 - val_acc: 0.9851
Epoch 2/10
60000/60000 [==============================] - 404s - loss: 0.0405 - acc: 0.9878 - val_loss: 0.0388 - val_acc: 0.9873
Epoch 3/10
60000/60000 [==============================] - 408s - loss: 0.0238 - acc: 0.9923 - val_loss: 0.0438 - val_acc: 0.9862
Epoch 4/10
60000/60000 [==============================] - 405s - loss: 0.0143 - acc: 0.9954 - val_loss: 0.0566 - val_acc: 0.9841
Epoch 5/10
60000/60000 [==============================] - 404s - loss: 0.0105 - acc: 0.9964 - val_loss: 0.0443 - val_acc: 0.9861
Epoch 6/10
60000/60000 [==============================] - 408s - loss: 0.0078 - acc: 0.9973 - val_loss: 0.0451 - val_acc: 0.9872
Epoch 7/10
60000/60000 [==============================] - 637s - loss: 0.0075 - acc: 0.9976 - val_loss: 0.0486 - val_acc: 0.9868
Epoch 8/10
60000/60000 [=======================

## エポックごとのロスを確認しよう

In [ ]:
plt.plot(fit.history['loss'])
plt.plot(fit.history['val_loss'])
plt.title('model sparse categorical crossentropy accuracy')
plt.ylabel('sparse categorical crossentropy accuracy')
plt.xlabel('epoch')
plt.legend(['training data', 'validation data'], loc='upper right')
plt.show()

## エポックごとの正解率を確認しよう

In [ ]:
plt.plot(fit.history['acc'])
plt.plot(fit.history['val_acc'])
plt.title('model sparse categorical crossentropy accuracy')
plt.ylabel('sparse categorical crossentropy accuracy')
plt.xlabel('epoch')
plt.legend(['training data', 'validation data'], loc='upper right')
plt.show()

## モデルを評価しよう

In [ ]:
# 精度を評価します。戻り値はそれぞれloss、accuracyです。
loss, accuracy = model.evaluate(X_test, y_test)
print('loss', loss)
print('accuracy', accuracy)